In [1]:
#Aplicacion de los mejores hiperparametros encontrados en una bayesiana
#Utilizando clase_binaria =  [  SI = { "BAJA+1", "BAJA+2"} ,  NO="CONTINUA ]

#cargo las librerias que necesito
require("data.table")
require("rpart")
require("rpart.plot")


#Aqui se debe poner la carpeta de la materia de SU computadora local
setwd("C:\\Users\\jesia\\Desktop\\4_DMEyF\\")  #Establezco el Working Directory

Loading required package: data.table

Loading required package: rpart

Loading required package: rpart.plot



In [2]:
#cargo el dataset
dataset  <- fread("./datasets/competencia1_2022.csv" )
#elimino por data drifting
#dataset[,Visa_fultimo_cierre:=NULL]
#dataset[,Master_fultimo_cierre:=NULL]

In [3]:
nrow( dataset )
ncol( dataset )

[1] 324242

[1] 155

In [4]:


#creo la clase_binaria SI={ BAJA+1, BAJA+2 }    NO={ CONTINUA }
dataset[ foto_mes==202101, 
         clase_binaria :=  ifelse( clase_ternaria=="CONTINUA", "NO", "SI" ) ]


dtrain  <- dataset[ foto_mes==202101 ]  #defino donde voy a entrenar
dapply  <- dataset[ foto_mes==202103 ]  #defino donde voy a aplicar el modelo




In [5]:
# Entreno el modelo
# obviamente rpart no puede ve  clase_ternaria para predecir  clase_binaria
#  #no utilizo Visa_mpagado ni  mcomisiones_mantenimiento por drifting

modelo  <- rpart(formula=   "clase_binaria ~ . -clase_ternaria",
                 data=      dtrain,  #los datos donde voy a entrenar
                 xval=         0,
                 cp= -0.528,
                 minsplit=  1000, #ajuste manual de minsplit. 
                 minbucket=  264,   
                 maxdepth=     10 ) #ajuste manual de maxdepth. 

#Lo mejor que obtuve por opt bayesiana fue: cp=-0.13717, min_split=1683, min_bucket=264, max_depth=18
feature_imp <- head(as.data.frame(modelo$variable.importance), 20)
feature_imp


,modelo$variable.importance
,<dbl>
ctrx_quarter,119.043268
mcuentas_saldo,43.445362
cdescubierto_preacordado,38.446236
active_quarter,30.642834
mcuenta_corriente,30.054549
cprestamos_personales,29.981218
mprestamos_personales,29.227296
mcaja_ahorro,27.508217
ccomisiones_otras,21.560275


In [28]:
#----------------------------------------------------------------------------
# habilitar esta seccion si el Fiscal General  Alejandro Bolaños  lo autoriza
#----------------------------------------------------------------------------

# corrijo manualmente el drifting de  Visa_fultimo_cierre
dapply[ Visa_fultimo_cierre== 1, Visa_fultimo_cierre :=  4 ]
dapply[ Visa_fultimo_cierre== 7, Visa_fultimo_cierre := 11 ]
dapply[ Visa_fultimo_cierre==21, Visa_fultimo_cierre := 25 ]
dapply[ Visa_fultimo_cierre==14, Visa_fultimo_cierre := 18 ]
dapply[ Visa_fultimo_cierre==28, Visa_fultimo_cierre := 32 ]
dapply[ Visa_fultimo_cierre==35, Visa_fultimo_cierre := 39 ]
dapply[ Visa_fultimo_cierre> 39, Visa_fultimo_cierre := Visa_fultimo_cierre + 4 ]

# corrijo manualmente el drifting de  Visa_fultimo_cierre
dapply[ Master_fultimo_cierre== 1, Master_fultimo_cierre :=  4 ]
dapply[ Master_fultimo_cierre== 7, Master_fultimo_cierre := 11 ]
dapply[ Master_fultimo_cierre==21, Master_fultimo_cierre := 25 ]
dapply[ Master_fultimo_cierre==14, Master_fultimo_cierre := 18 ]
dapply[ Master_fultimo_cierre==28, Master_fultimo_cierre := 32 ]
dapply[ Master_fultimo_cierre==35, Master_fultimo_cierre := 39 ]
dapply[ Master_fultimo_cierre> 39, Master_fultimo_cierre := Master_fultimo_cierre + 4 ]


In [29]:
#aplico el modelo a los datos nuevos
prediccion  <- predict( object=  modelo,
                        newdata= dapply,
                        type = "prob")

#prediccion es una matriz con DOS columnas, llamadas "NO", "SI"
#cada columna es el vector de probabilidades 



In [30]:
#agrego a dapply una columna nueva que es la probabilidad de BAJA+2
dfinal  <- copy( dapply[ , list(numero_de_cliente) ] )
dfinal[ , prob_SI := prediccion[ , "SI"] ]


# por favor cambiar por una semilla propia
# que sino el Fiscal General va a impugnar la prediccion
set.seed(124739)  
dfinal[ , azar := runif( nrow(dapply) ) ]

# ordeno en forma descentente, y cuando coincide la probabilidad, al azar
setorder( dfinal, -prob_SI, azar )




In [32]:
dir.create( "./exp/" )
dir.create( "./exp/competencia_1" )


for( corte  in  c( 7500, 8000, 8500, 9000, 9500, 10000, 10500, 11000 ) )
{
  #le envio a los  corte  mejores,  de mayor probabilidad de prob_SI
  dfinal[ , Predicted := 0L ]
  dfinal[ 1:corte , Predicted := 1L ]


  fwrite( dfinal[ , list(numero_de_cliente, Predicted) ], #solo los campos para Kaggle
           file= paste0( "./exp/competencia_1/v5_ob",  corte, ".csv"),
           sep=  "," )
}

Warning message in dir.create("./exp/"):
"'.\exp' already exists"
Warning message in dir.create("./exp/resignacion_v2"):
"'.\exp\resignacion_v2' already exists"


In [ ]:
v1- original
v2 - descomentando
v3- modif de original x mis hiperparam
v4- con hip modificados a mano 